<a href="https://colab.research.google.com/github/ro7hitkumar/chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
!pip install transformers
!pip install wikipedia



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torc

In [2]:
import pandas as pd
import gradio as gr
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import wikipedia

# Define the responses (Expanded)
FALLBACK_RESPONSES = {
    "hello": "Hi there!",
    "hi": "Hi there how may I help you",
    "hey": "Hi there \nhow may I help you",
    "how are you?": "I'm doing great, thanks! How about you?",
    "what is your name?": "I am a chatbot! You can call me ChatBot Transformer.",
    "tell me a joke": "Why did the scarecrow win an award? Because he was outstanding in his field!",
    "what is ai?": "AI stands for Artificial Intelligence, the simulation of human intelligence by machines.",
    "what is your favorite color?": "I don't see colors, but if I could, I'd probably like blue!",
    "how old are you?": "I was created just recently, so I'm quite young!",
    "tell me a fun fact": "Did you know? Honey never spoils. Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3000 years old!",
    "who created you?": "I was created by a programmer who loves AI and innovation!",
    "what is the weather today?": "I'm not connected to live weather data, but I hope it's sunny where you are!",
    "what is admission eligibility criteria?": "Eligibility: For undergraduate programs, a minimum of 50% in 12th grade is required. For postgraduate programs, a bachelor's degree is mandatory.",
    "what is admission process?": "The admission process involves applying online through our portal. You need to fill out the application form, upload the necessary documents, and pay the application fee.",
    "what are the admission dates?": "Admissions commence on April 1st and close on June 30th.",
    "Which exams are accepted for exams?": "We accept scores from JEE, CAT, and our own entrance examination.",
    "what are the ug programs?": "Available undergraduate programs include B.Tech, B.Sc, BCA, and BA.",
    "what are the pg programs?": "Available postgraduate programs include M.Tech, M.Sc, MCA, and MA.",
    "what are the diploma programs?": "We offer diplomas in Digital Marketing, Data Science, and other fields.",
    "what are the research programs?": "Research areas include Computer Science, Physics, and Management.",
    "give me fees structure": "The fee structure is as follows: Undergraduate programs start at $10,000 per year, and postgraduate programs start at $15,000 per year.",
    "what are the fees payment methods?": "Payments can be made via Net Banking, Credit/Debit Card, or UPI.",
    "scholarships offered by college": "We offer merit-based and need-based scholarships.",
    "what is refund policy?": "Refunds are allowed within 30 days with a 10% deduction.",
    "what are library facilities?": "Our library houses over 50,000 books and e-resources.",
    "what are hostel facilities?": "We have separate hostels for boys and girls, equipped with modern amenities.",
    "what are sports facilities?": "We offer facilities for cricket, football, a gym, and more.",
    "what are transport facilities?": "Bus services are available across the city.",
    "placements statistics": "We have a 95% placement rate, with an average package of $50,000 per year.",
    "placements recruiters": "Top recruiters include Google, Amazon, and Infosys.",
    "placements internships": "We offer internships with industry leaders.",
    "placements alumni": "Our alumni include top industry professionals and successful entrepreneurs.",
    "contact department": "For departmental inquiries, email admissions@college.com or call +123-456-7890.",
    "contact general": "For general inquiries, email info@college.com or call +123-456-0000.",
    "contact social": "Follow us on Facebook, Twitter, and Instagram.",
    "greetings": "Hello! How can I assist you today? You can ask me about admissions, programs, fees, facilities, placements, or contact information.",
    "admission":
        "Eligibility: UG requires 50% in 12th; PG requires a bachelor's degree.\n"
        "Apply online via our portal. Fill the form, upload documents, and pay the fee.\n"
        "Admissions start April 1 and close June 30.\n"
        "We accept JEE, CAT, and our entrance exam.\n"
        "For national admissions, you can contact us at 0120-4370000 or +91-9582847072. \n"
        "For international admissions, please call 0120-4806884 or +91-7838141101. \n"
        "You can also email us at admissions@galgotiasuniversity.edu.in. \n"
        "For more details, visit our admission helpline page: https://www.galgotiasuniversity.edu.in/p/admission/admission-helpline",
    "programs":
        "Available programs: B.Tech, B.Sc, BCA, BA.\n"
        "Available programs: M.Tech, M.Sc, MCA, MA.\n"
        "Diplomas in Digital Marketing, Data Science, etc.\n"
        "Research areas include Computer Science, Physics, Management.",
    "fees":
        "Fees: UG starts at $10,000/year; PG starts at $15,000/year.\n"
        "Pay via Net Banking, Credit/Debit Card, or UPI.\n"
        "Merit-based and need-based scholarships available.\n"
        "Refunds allowed within 30 days with a 10% deduction.",
    "facilities":
        "Library has 50,000+ books and e-resources.\n"
        "Separate hostels for boys and girls with modern amenities.\n"
        "Facilities for cricket, football, gym, and more.\n"
        "Bus services available across the city.",
    "placements":
        "95% placement rate. Avg package: $50,000/year.\n"
        "Top recruiters: Google, Amazon, Infosys.\n"
        "Internships offered with industry leaders.\n"
        "Alumni include top industry professionals and entrepreneurs.",
    "contact":
        "Email: admissions@college.com; Phone: +123-456-7890.\n"
        "Email: info@college.com; Phone: +123-456-0000.\n"
        "Follow us on Facebook, Twitter, Instagram.",
    "admission eligibility": "Eligibility of UG requires 50% & in 12th PG requires a bachelor's degree.",
    "admission process": "Apply online via our portal. Fill the form, upload documents, and pay the fee.",
    "admisiion dates": "Admissions start April 1 and close June 30.",
    "exams": "We accept JEE, CAT, and our entrance exam.",
    "undergraduate": "Available programs: B.Tech, B.Sc, BCA, BA.",
    "postgraduate": "Available programs: M.Tech, M.Sc, MCA, MA.",
    "diploma": "Diplomas in Digital Marketing, Data Science, etc.",
    "research": "Research areas include Computer Science, Physics, Management.",
    "what are the course offered by college?" :
        "We offer a variety of programs across multiple schools, including: "
        "- School of Computer Science & Engineering\n"
        "- School of Business\n"
        "- School of Law\n"
        "- School of Media & Communication Studies\n"
        "For a comprehensive list of programs, please visit: https://www.galgotiasuniversity.edu.in/p/programmes-and-fees",
    "give me the fees structure":
        "The fee structure varies by program. \n"
        "For detailed information, please visit our fee structure page: https://www.galgotiasuniversity.edu.in/p/admission/fee-structure",
    "give me payment methods": "Pay via Net Banking, Credit/Debit Card, or UPI.",
    "what are available scholarships?": "Merit-based and need-based scholarships available.",
    "what is refund process?": "Refunds allowed within 30 days with a 10% deduction.",
    "give me library facilities": "Library has 50,000+ books and e-resources.",
    "give me hostel facility details": "Separate hostels for boys and girls with modern amenities.",
    "sports facility details": "Facilities for cricket, football, gym, and more.",
    "transport": "Bus services available across the city.",
    "statistics": "95% placement rate. Avg package: $50,000/year.",
    "recruiters": "Top recruiters: Google, Amazon, Infosys.",
    "internships": "Internships offered with industry leaders.",
    "alumni": "Alumni include top industry professionals and entrepreneurs.",
    "department": "Email: admissions@college.com; Phone: +919582847072.",
    "give me contact details":
        "You can reach us at:\n"
        "Phone: 0120-4370000\n"
        "Phone: +91-9582847072\n"
        "Email: admissions@galgotiasuniversity.edu.in\n"
        "For more contact details, visit: https://www.galgotiasuniversity.edu.in/p/contact-us",
    "where to follow in social media?": "Follow us on Facebook, Twitter, Instagram.",
    "give me faculty details":
        "Galgotias University boasts experienced faculty members across various disciplines. \n"
        "For more information, you can visit: https://galgotiacollege.edu/gcop/faculty",
    "give me social media links of college":
        "Facebook : https://www.facebook.com/GalgotiasUniversity \n"
        "Twitter : https://x.com/GalgotiasGU?mx=2 \n"
        "Instagram : https://www.instagram.com/galgotias_university/ "
}


In [3]:


# Dataset Definition
class ChatDataset(Dataset):
    def __init__(self, questions, answers, tokenizer, max_length):
        self.questions = questions
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        answer = self.answers[idx]

        encoded_question = self.tokenizer(
            question, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt"
        )
        encoded_answer = self.tokenizer(
            answer, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt"
        )

        return encoded_question["input_ids"].squeeze(0), encoded_answer["input_ids"].squeeze(0)

# Transformer Model Definition
class SimpleTransformerChatbot(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads, num_layers, hidden_size):
        super(SimpleTransformerChatbot, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.transformer = nn.Transformer(embed_size, num_heads, num_layers, dim_feedforward=hidden_size, batch_first=True)
        self.fc = nn.Linear(embed_size, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x, x)
        x = self.fc(x)
        return x

# Data Preparation
questions = ["Hello", "How are you?", "What is your name?", "Tell me a joke", "What is AI?", "What is your favorite color?", "what are the ug programs?", "How old are you?", "Tell me a fun fact", "Who created you?", "What is the weather today?"]
answers = list(FALLBACK_RESPONSES.values())

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
max_length = 20

dataset = ChatDataset(questions, answers, tokenizer, max_length)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Model, Loss, and Optimizer
vocab_size = tokenizer.vocab_size
embed_size = 128
num_heads = 4
num_layers = 2
hidden_size = 256

model = SimpleTransformerChatbot(vocab_size, embed_size, num_heads, num_layers, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
def train_model():
    model.train()
    for epoch in range(20):  # Train for more epochs
        total_loss = 0
        for questions, answers in dataloader:
            optimizer.zero_grad()
            outputs = model(questions)
            loss = criterion(outputs.view(-1, vocab_size), answers.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")

train_model()




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Epoch 1, Loss: 9.285717089970907
Epoch 2, Loss: 7.897238413492839
Epoch 3, Loss: 6.903099695841472
Epoch 4, Loss: 5.81614089012146
Epoch 5, Loss: 5.092512448628743
Epoch 6, Loss: 4.372107783953349
Epoch 7, Loss: 4.247229655583699
Epoch 8, Loss: 3.7781222661336265
Epoch 9, Loss: 3.7747623523076377
Epoch 10, Loss: 3.688126047452291
Epoch 11, Loss: 3.5714096625645957
Epoch 12, Loss: 3.5962957541147866
Epoch 13, Loss: 3.4871605237325034
Epoch 14, Loss: 3.5848109126091003
Epoch 15, Loss: 3.4971832434336343
Epoch 16, Loss: 3.4541743993759155
Epoch 17, Loss: 3.4730082750320435
Epoch 18, Loss: 3.366425116856893
Epoch 19, Loss: 3.1306045850118003
Epoch 20, Loss: 3.158478538195292


In [ ]:

# Gradio Interface with Wikipedia Integration
def chatbot_response(user_input):
    user_input = user_input.lower().strip()

    # Check fallback responses
    if user_input in FALLBACK_RESPONSES:
        return FALLBACK_RESPONSES[user_input]

    # Check if the user is asking "Who is X?"
    if user_input.startswith("who is"):
        search_term = user_input.replace("who is", "").strip()
        try:
            summary = wikipedia.summary(search_term, sentences=2)
            return summary
        except wikipedia.exceptions.DisambiguationError as e:
            return f"Your query is too broad. Did you mean one of these? {e.options[:5]}"
        except wikipedia.exceptions.PageError:
            return "Sorry, I couldn't find anything on that topic."

    # Model Prediction
    model.eval()
    with torch.no_grad():
        encoded_input = tokenizer(
            user_input, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt"
        )
        input_ids = encoded_input["input_ids"]
        outputs = model(input_ids)
        predicted_ids = outputs.argmax(dim=-1).squeeze(0).tolist()
        decoded_output = tokenizer.decode(predicted_ids, skip_special_tokens=True)
        return decoded_output if decoded_output.strip() else "Sorry, I couldn't generate a response."

interface = gr.Interface(fn=chatbot_response, inputs="text", outputs="text", title="Advanced Conversational AI: Enhancing GU-Enquiry Chatbot with NLP and Machine Learning", description="A comprehensive chatbot leveraging NLP, Machine Learning, and Wikipedia integration for intelligent and dynamic conversations.")
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://80d2cc83cbdbe3597c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
